In [1]:
import gc
import os
import math
import pickle

import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from operator import itemgetter

from sklearn.utils import shuffle
from collections import defaultdict

# from metric import PrintMetric

import warnings
warnings.filterwarnings("ignore")

raw_data_path = '../rank/examples/dataset/raw_data_sample'
new_data_path = '../rank/examples/dataset/new_data_sample'

os.makedirs(new_data_path, exist_ok=True)

/home/harry/miniconda3/envs/news_rec_py3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
user_info = pd.read_csv(raw_data_path + '/user_info.csv', header=None)
user_info.columns = ["user_id", "设备名称", "操作系统", "所在省", "所在市", "年龄","性别"]

user_info.head()

,user_id,设备名称,操作系统,所在省,所在市,年龄,性别
0,1000652892,PACM00,Android,河北,唐山,"A_0_24:0.615458,A_25_29:0.086233,A_30_39:0.141...","female:0.280295,male:0.719705"
1,1000908852,MI6X,Android,上海,上海,"A_0_24:0.123255,A_25_29:0.208225,A_30_39:0.298...","female:0.000000,male:1.000000"
2,1001168798,iPhone11,IOS,NaN,NaN,"A_0_24:0.436296,A_25_29:0.489370,A_30_39:0.061...","female:0.870710,male:0.129290"
3,1001305614,M2103K19C,Android,江苏,苏州,"A_0_24:0.006632,A_25_29:0.043408,A_30_39:0.350...","female:0.000000,male:1.000000"
4,1001309700,LYA-AL10,Android,黑龙江,哈尔滨,"A_0_24:0.413768,A_25_29:0.437501,A_30_39:0.141...","female:0.000000,male:1.000000"


In [3]:
doc_info = pd.read_csv(raw_data_path + '/doc_info.csv', header=None)
doc_info.columns = ["item_id", "标题", "发文时间", "图片数量",  "一级分类", "二级分类", "关键词"]

doc_info.head()

,item_id,标题,发文时间,图片数量,一级分类,二级分类,关键词
0,361653323,"疫情谣言粉碎机丨接种新冠疫苗后用麻药或致死?盘点最新疫情谣言,别被忽悠了",1624522285000,1,健康,健康/疾病防护治疗及西医用药,"医生:14.760494,吸烟:16.474872,板蓝根:15.597788,板蓝根^^熏..."
1,426732705,"实拍本田飞度:空间真大,8万出头工薪族可选,但内饰能忍?",1610808303000,9,汽车,汽车/买车,"155n:8.979802,polo:7.951116,中控台:5.954278,中网:7...."
2,430221183,搭载135kw电机比亚迪秦plus纯电动版外观更精致,1612581556000,2,汽车,汽车/买车,"etc:12.055207,代表:8.878175,内饰:5.342025,刀片:9.453..."
3,441756326,【提车作业】不顾他人眼光帕萨特phev俘获30老男人浪子心,1618825835000,23,汽车,汽车/买车,"丰田凯美瑞:12.772149,充电器:8.394001,品牌:8.436843,城市:7...."
4,443485341,"魏延有反骨之心都能重用,赵云忠心为什么却不被重用?",1619484501000,4,历史,历史/中国史,"三国:8.979797,五虎将:13.072728,人才:7.532783,保镖:6.811..."


In [4]:
all_data = pd.read_csv(raw_data_path + '/train_data.csv', header=None)
all_data.columns = ["user_id", "item_id", "展现时间", "网路环境", "刷新次数", "展现位置", "是否点击", "消费时长（秒）"]

all_data.head()

,user_id,item_id,展现时间,网路环境,刷新次数,展现位置,是否点击,消费时长（秒）
0,1000014754,463510256,1624843756147,5,0,16,0,0
1,1000014754,463852707,1624843756147,5,0,13,1,80
2,1000014754,464757134,1625052999841,5,0,13,1,1050
3,1000014754,464617167,1625052999841,5,0,16,1,286
4,1000014754,465426190,1625382421168,5,0,5,0,0


In [5]:
all_data['展现时间'] = all_data['展现时间'].astype('str')
all_data['展现时间'] = all_data['展现时间'].apply(lambda x: int(x[:10]))

all_data['展现时间'] = pd.to_datetime(all_data['展现时间'], unit='s', errors='coerce')
all_data['展现时间_日期'] = all_data['展现时间'].dt.day

all_data.head()

,user_id,item_id,展现时间,网路环境,刷新次数,展现位置,是否点击,消费时长（秒）,展现时间_日期
0,1000014754,463510256,2021-06-28 01:29:16,5,0,16,0,0,28
1,1000014754,463852707,2021-06-28 01:29:16,5,0,13,1,80,28
2,1000014754,464757134,2021-06-30 11:36:39,5,0,13,1,1050,30
3,1000014754,464617167,2021-06-30 11:36:39,5,0,16,1,286,30
4,1000014754,465426190,2021-07-04 07:07:01,5,0,5,0,0,4


In [6]:
mode = 'debug'

if mode == 'debug':
    all_data = shuffle(all_data)
    all_data.reset_index(drop=True)

    train_data = all_data[(all_data['展现时间_日期'] >= 5) & (all_data['展现时间_日期'] < 6)]
    test_data = all_data.loc[all_data['展现时间_日期'] == 6, :]
else:
    train_data = all_data[(all_data['展现时间_日期'] >= 1) & (all_data['展现时间_日期'] < 6)]
    test_data = all_data.loc[all_data['展现时间_日期'] == 6, :]

del all_data, doc_info, user_info
gc.collect()

75

In [7]:
print(train_data['user_id'].nunique())
print(train_data['item_id'].nunique())

15655
33664


In [8]:
class UserCF(object):
    def __init__(self, his_data):
        self.user_set = set()
        self.item_set = set()

        self.his_data = his_data
        self.user_sim_matrix = dict()
        self.user_interacted_num = defaultdict(int)
        self.item_interacted_num = defaultdict(int)     # 热门推荐时会用到

    def calculate_similarity_matrix(self):
        item2users = self.his_data.groupby('item_id')['user_id'].apply(list).reset_index()

        # print(f'计算ItemCF第一阶段...')
        pbar = tqdm(total=item2users.shape[0])
        for idx, row in item2users.iterrows():
            self.item_set.add(row['item_id'])
            self.user_set.update(row['user_id'])
            self.item_interacted_num[row['item_id']] += len(row['user_id'])
            for idx1, user_1 in enumerate(row['user_id']):
                self.user_interacted_num[user_1] += 1
                self.user_sim_matrix.setdefault(user_1, {})
                for idx2, user_2 in enumerate(row['user_id']):
                    if user_1 == user_2:
                        continue
                    self.user_sim_matrix[user_1].setdefault(user_2, 0)
                    # 热门物品用在计算用户之间相似度时，贡献小于非热门物品
                    self.user_sim_matrix[user_1][user_2] += 1 / math.log(1 + len(row['user_id']))
            pbar.update(1)
        pbar.close()

        # 理论上，用户之间共现的物品越多，相似度越高
        # 但是，活跃用户与很多用户之间的相似度都很高
        print(f'计算UserCF第二阶段...')
        for user_1, related_users in tqdm(self.user_sim_matrix.items()):
            for user_2, weight in related_users.items():
                # 打压活跃用户
                self.user_sim_matrix[user_1][user_2] =\
                    weight / math.sqrt(self.user_interacted_num[user_1] * self.user_interacted_num[user_2])

    def __call__(self, users, _n=50, _topk=20):
        print(f'开始ItemCF召回: Recall@{topk}-Near@{_n}')
        user2items = self.his_data.groupby('user_id')['item_id'].apply(list)
        popular_items = [val[0] for val in sorted(
            self.item_interacted_num.items(), key=lambda x: x[1], reverse=True)[:_topk]]

        user_rec = {}
        for user_id in tqdm(users):
            # 新用户，直接推荐热门物品
            if user_id not in self.user_set:
                user_rec[user_id] = popular_items
            else:
                rank = defaultdict(int)
                for relate_user, user_smi_score in sorted(self.user_sim_matrix[user_id].items(),
                                                      key=itemgetter(1), reverse=True)[:_n]:
                    for candidate_item in user2items.loc[relate_user]:
                        # if candidate_item in user2items.loc[user_id]:
                        #     continue
                        rank[candidate_item] += user_smi_score
                rec_items = [item[0] for item in sorted(rank.items(), key=itemgetter(1), reverse=True)[:_topk]]
                # 如果推荐的物品不够，用热门物品进行填充
                rec_items += popular_items[:topk-len(rec_items)]
                user_rec[user_id] = rec_items

        return user_rec

In [9]:
ucf_cls_path = os.path.join(new_data_path, 'user_cf')
os.makedirs(ucf_cls_path, exist_ok=True)

In [10]:
demo_ucf_path = os.path.join(ucf_cls_path, mode+'_ufc.pkl')

if os.path.exists(demo_ucf_path):
    with open(demo_ucf_path, 'rb') as file:
        demo_ucf = pickle.loads(file.read())
        file.close()
else:
    demo_ucf = UserCF(train_data)
    demo_ucf.calculate_similarity_matrix()
    demo_ucf_pkl = pickle.dumps(demo_ucf)

    output_ucf = open(demo_ucf_path, 'wb')
    output_ucf.write(demo_ucf_pkl)
    output_ucf.close()

In [11]:
n, topk = 50, 100

# 召回
test_users = test_data['user_id'].unique()
icf_rec_result = demo_ucf(test_users, n, topk)

test_user_group = test_data.groupby('user_id')['item_id'].agg(list).reset_index()
test_pred = [icf_rec_result[user_id] for user_id in test_user_group['user_id']]
test_true = test_user_group['item_id'].to_list()

开始ItemCF召回: Recall@100-Near@50


  0%|          | 0/13792 [00:00<?, ?it/s]

In [12]:
PrintMetric(test_true, test_pred, topk)

MAP@100:  0.012998163356723474
Recall@100:  0.14660816973054847
Precision@100:  0.02628625290023202
F1@100:  0.036618405618781665
